In [ ]:
import os
import requests
import base64
import json
import uuid

def download_images_from_files(json_file_paths, download_folder="img_json"):
    """
    Descarga imágenes desde varios archivos JSON que contienen URLs y datos base64.

    :param json_file_paths: Lista de rutas a archivos JSON con datos de imágenes.
    :param download_folder: Carpeta donde se guardarán las imágenes.
    """
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)  # Crear carpeta si no existe

    for file_path in json_file_paths:
        print(f"Procesando archivo: {file_path}")

        # Leer datos del archivo JSON
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                json_data = json.load(file)
        except Exception as e:
            print(f"Error al leer el archivo {file_path}: {e}")
            continue

        for idx, item in enumerate(json_data):
            image_data = item.get("Imagen2")
            if not image_data:
                print(f"Item {idx + 1} en {file_path} no contiene datos de imagen.")
                continue

            unique_id = uuid.uuid4().hex  # Generar un identificador único

            if image_data.startswith("http"):
                # Procesar URL de imagen
                try:
                    response = requests.get(image_data, stream=True)
                    response.raise_for_status()  # Asegura que la respuesta es válida

                    # Extraer extensión de la imagen
                    file_extension = image_data.split(".")[-1].split('?')[0]
                    if file_extension not in ["jpg", "jpeg", "png", "gif", "webp"]:
                        file_extension = "jpg"  # Default

                    file_name = f"image_{unique_id}.{file_extension}"
                    file_path = os.path.join(download_folder, file_name)

                    # Guardar la imagen
                    with open(file_path, "wb") as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)

                    print(f"Imagen descargada desde URL: {file_path}")
                except requests.exceptions.RequestException as e:
                    print(f"Error al descargar desde URL en item {idx + 1}: {e}")

            elif image_data.startswith("data:image"):
                # Procesar imagen codificada en base64
                try:
                    # Obtener datos base64 sin el encabezado
                    header, encoded = image_data.split(",", 1)
                    file_extension = header.split(";")[0].split("/")[-1]  # Obtener formato de imagen
                    file_name = f"image_{unique_id}_base64.{file_extension}"
                    file_path = os.path.join(download_folder, file_name)

                    # Decodificar y guardar
                    with open(file_path, "wb") as file:
                        file.write(base64.b64decode(encoded))

                    print(f"Imagen guardada desde base64: {file_path}")
                except Exception as e:
                    print(f"Error al procesar base64 en item {idx + 1}: {e}")

            else:
                print(f"Formato no soportado en item {idx + 1}")

# Uso del script
# Lista de archivos JSON
json_files = [
    "C:/Users/Leonardo/Downloads/img31.json",
    "C:/Users/Leonardo/Downloads/img32.json",
    "C:/Users/Leonardo/Downloads/img33.json",
    "C:/Users/Leonardo/Downloads/img34.json",
    "C:/Users/Leonardo/Downloads/img35.json",
    "C:/Users/Leonardo/Downloads/img36.json",
    "C:/Users/Leonardo/Downloads/img37.json",
    "C:/Users/Leonardo/Downloads/img38.json",
    "C:/Users/Leonardo/Downloads/img39.json",
]

download_images_from_files(json_files)

In [ ]:
### VERSION 1
import os
from PIL import Image, ImageEnhance, ImageOps
import cv2
import numpy as np
from tqdm import tqdm

class ImageProcessor:
    def __init__(self, input_folder, output_folder):
        """
        Inicializa el procesador de imágenes.
        
        Args:
            input_folder (str): Ruta de la carpeta con imágenes originales
            output_folder (str): Ruta donde se guardarán las imágenes procesadas
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
        
    def create_output_dirs(self):
        """Crea los directorios necesarios para cada versión."""
        os.makedirs(self.output_folder, exist_ok=True)
        for i in range(1, 6):
            os.makedirs(os.path.join(self.output_folder, f"version_{i}"), exist_ok=True)
            os.makedirs(os.path.join(self.output_folder, f"version_{i}_mirror"), exist_ok=True)

    def apply_morning_filter(self, image):
        """
        Aplica filtro para simular luz de mañana soleada (9:00 AM).
        Aumenta brillo y temperatura de color, mantiene buen contraste.
        """
        # Convierte a array numpy para manipulación
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (más cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=10)
        
        # Aumenta los tonos azules ligeramente
        b, g, r = cv2.split(img_array)
        b = cv2.convertScaleAbs(b, alpha=1.1, beta=0)
        img_array = cv2.merge([b, g, r])
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajusta brillo y contraste
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.1)
        
        return image

    def apply_late_afternoon_filter(self, image):
        """
        Aplica filtro para simular luz de tarde (5:00 PM).
        Añade tonos cálidos y sombras más pronunciadas.
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (más cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Aumenta los tonos rojos y reduce los azules
        b, g, r = cv2.split(img_array)
        r = cv2.convertScaleAbs(r, alpha=1.15, beta=10)
        b = cv2.convertScaleAbs(b, alpha=0.9, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Ajusta el contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=0)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajusta saturación y contraste
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        
        return image

    def apply_cloudy_afternoon_filter(self, image):
        """
        Aplica filtro para simular luz de tarde nublada (5:00 PM).
        Reduce contraste y saturación, aumenta difusión.
        """
        # Ajusta saturación y brillo
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.8)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.9)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(0.85)
        
        # Aplica un ligero desenfoque para simular difusión
        img_array = np.array(image)
        img_array = cv2.GaussianBlur(img_array, (3, 3), 0)
        image = Image.fromarray(img_array)
        
        return image

    def apply_night_filter(self, image):
        """
        Aplica filtro para simular luz nocturna (10:00 PM).
        Reduce brillo, añade tono azulado y aumenta sombras.
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (más fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Aumenta los tonos azules y reduce los rojos
        b, g, r = cv2.split(img_array)
        b = cv2.convertScaleAbs(b, alpha=1.1, beta=10)
        r = cv2.convertScaleAbs(r, alpha=0.85, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Reduce el brillo general
        img_array = cv2.convertScaleAbs(img_array, alpha=0.7, beta=-10)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajusta contraste y saturación
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.8)
        
        return image

    def create_mirror_version(self, image):
        """Crea versión espejo de la imagen."""
        return ImageOps.mirror(image)

    def save_image(self, image, version, filename, is_mirror=False):
        """Guarda la imagen procesada en el directorio correspondiente."""
        subfolder = f"version_{version}_mirror" if is_mirror else f"version_{version}"
        output_path = os.path.join(self.output_folder, subfolder, filename)
        image.save(output_path, quality=95, subsampling=0)

    def process_single_image(self, filename):
        """Procesa una única imagen y genera todas sus versiones."""
        input_path = os.path.join(self.input_folder, filename)
        
        try:
            # Abre la imagen original
            original = Image.open(input_path).convert('RGB')
            
            # Lista de funciones de procesamiento para cada versión
            processors = [
                lambda x: x,  # Versión 1: Original
                self.apply_morning_filter,  # Versión 2: Mañana soleada
                self.apply_late_afternoon_filter,  # Versión 3: Tarde
                self.apply_cloudy_afternoon_filter,  # Versión 4: Tarde nublada
                self.apply_night_filter,  # Versión 5: Noche
            ]
            
            # Procesa y guarda cada versión
            for version, processor in enumerate(processors, 1):
                # Procesa la imagen
                processed = processor(original.copy())
                
                # Guarda versión normal y espejo
                self.save_image(processed, version, (str(version) + "_" + filename))
                self.save_image(self.create_mirror_version(processed), version, ("m" + str(version) + "_" + filename), True)
                
        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")

    def process_all_images(self):
        """Procesa todas las imágenes en el directorio de entrada."""
        # Crea directorios de salida
        self.create_output_dirs()
        
        # Obtiene lista de imágenes
        images = [f for f in os.listdir(self.input_folder) 
                 if os.path.splitext(f)[1].lower() in self.valid_extensions]
        
        if not images:
            print("No se encontraron imágenes para procesar.")
            return
        
        print(f"Procesando {len(images)} imágenes...")
        
        # Procesa cada imagen con barra de progreso
        for filename in tqdm(images):
            self.process_single_image(filename)
            
        print("\n¡Procesamiento completado!")

# Ejemplo de uso
def main():
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/test/images"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/test/images_processed"
    
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()

if __name__ == "__main__":
    main()

In [ ]:
### VERSION 2
import os
from PIL import Image, ImageEnhance, ImageOps
import cv2
import numpy as np
from tqdm import tqdm

class ImageProcessor:
    def __init__(self, input_folder, output_folder):
        """
        Inicializa el procesador de imágenes.
        
        Args:
            input_folder (str): Ruta de la carpeta con imágenes originales
            output_folder (str): Ruta donde se guardarán las imágenes procesadas
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
        
    def create_output_dirs(self):
        """Crea los directorios necesarios para cada versión."""
        os.makedirs(self.output_folder, exist_ok=True)
        for i in range(1, 6):
            os.makedirs(os.path.join(self.output_folder, f"version_{i}"), exist_ok=True)
            os.makedirs(os.path.join(self.output_folder, f"version_{i}_mirror"), exist_ok=True)
    
    def apply_morning_filter(self, image):
        """
        Filtro de mañana soleada (9:00 AM).
        Partiendo de iluminación de estudio, ajusta para simular luz natural de mañana:
        - Aumenta temperatura de color ligeramente
        - Añade sombras suaves pero definidas
        - Mantiene buena exposición general
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (ligeramente cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajuste sutil de canales para luz de mañana
        b, g, r = cv2.split(img_array)
        # Reduce azules ligeramente
        b = cv2.convertScaleAbs(b, alpha=0.95, beta=0)
        # Aumenta rojos suavemente
        r = cv2.convertScaleAbs(r, alpha=1.05, beta=5)
        img_array = cv2.merge([b, g, r])
        
        # Ajuste general de contraste y brillo
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=5)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales de imagen
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.1)
        
        return image

    def apply_late_afternoon_filter(self, image):
        """
        Aplica filtro para simular luz de tarde (5:00 PM).
        Añade tonos cálidos y sombras más pronunciadas.
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (más cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Aumenta los tonos rojos y reduce los azules
        b, g, r = cv2.split(img_array)
        r = cv2.convertScaleAbs(r, alpha=1.15, beta=10)
        b = cv2.convertScaleAbs(b, alpha=0.9, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Ajusta el contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=0)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajusta saturación y contraste
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        
        return image

    def apply_cloudy_afternoon_filter(self, image):
        """
        Filtro de tarde nublada (5:00 PM).
        Simula luz difusa de día nublado:
        - Reduce el contraste general
        - Enfatiza los tonos medios
        - Añade una sutil dominante fría
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (ligeramente fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajustes de canales para luz difusa
        b, g, r = cv2.split(img_array)
        # Aumenta azules sutilmente
        b = cv2.convertScaleAbs(b, alpha=1.05, beta=5)
        # Reduce rojos ligeramente
        r = cv2.convertScaleAbs(r, alpha=0.95, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Reduce contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=0.9, beta=10)
        
        # Aplica desenfoque sutil para simular dispersión
        img_array = cv2.GaussianBlur(img_array, (3, 3), 0.5)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales para ambiente nublado
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(0.9)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.85)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.95)
        
        return image

    def apply_night_filter(self, image):
        """
        Filtro nocturno (10:00 PM).
        Simula iluminación artificial nocturna:
        - Reduce significativamente el brillo
        - Añade tono azulado de luz nocturna
        - Aumenta las sombras
        - Simula iluminación artificial
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajustes de canales para luz nocturna
        b, g, r = cv2.split(img_array)
        # Enfatiza azules
        b = cv2.convertScaleAbs(b, alpha=1.15, beta=10)
        # Reduce rojos
        r = cv2.convertScaleAbs(r, alpha=0.8, beta=-10)
        # Ajusta verdes
        g = cv2.convertScaleAbs(g, alpha=0.9, beta=-5)
        img_array = cv2.merge([b, g, r])
        
        # Reducción general de luminosidad
        img_array = cv2.convertScaleAbs(img_array, alpha=0.75, beta=-20)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales para ambiente nocturno
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.7)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.3)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.85)
        
        return image

    def create_mirror_version(self, image):
        """Crea versión espejo de la imagen."""
        return ImageOps.mirror(image)

    def save_image(self, image, version, filename, is_mirror=False):
        """Guarda la imagen procesada en el directorio correspondiente."""
        subfolder = f"version_{version}_mirror" if is_mirror else f"version_{version}"
        output_path = os.path.join(self.output_folder, subfolder, filename)
        image.save(output_path, quality=95, subsampling=0)

    def process_single_image(self, filename):
        """Procesa una única imagen y genera todas sus versiones."""
        input_path = os.path.join(self.input_folder, filename)
        
        try:
            # Abre la imagen original
            original = Image.open(input_path).convert('RGB')
            
            # Lista de funciones de procesamiento para cada versión
            processors = [
                lambda x: x,  # Versión 1: Original
                self.apply_morning_filter,  # Versión 2: Mañana soleada
                self.apply_late_afternoon_filter,  # Versión 3: Tarde
                self.apply_cloudy_afternoon_filter,  # Versión 4: Tarde nublada
                self.apply_night_filter,  # Versión 5: Noche
            ]
            
            # Procesa y guarda cada versión
            for version, processor in enumerate(processors, 1):
                # Procesa la imagen
                processed = processor(original.copy())
                
                # Guarda versión normal y espejo
                self.save_image(processed, version, (str(version) + "_" + filename))
                self.save_image(self.create_mirror_version(processed), version, ("m" + str(version) + "_" + filename), True)
                
        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")

    def process_all_images(self):
        """Procesa todas las imágenes en el directorio de entrada."""
        # Crea directorios de salida
        self.create_output_dirs()
        
        # Obtiene lista de imágenes
        images = [f for f in os.listdir(self.input_folder) 
                 if os.path.splitext(f)[1].lower() in self.valid_extensions]
        
        if not images:
            print("No se encontraron imágenes para procesar.")
            return
        
        print(f"Procesando {len(images)} imágenes...")
        
        # Procesa cada imagen con barra de progreso
        for filename in tqdm(images):
            self.process_single_image(filename)
            
        print("\n¡Procesamiento completado!")



# Ejemplo de uso
def main():
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/ford_fiesta_zetec/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/dataset/test/ford_fiesta_zetec/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    # TODO: Añadir función que al ser llamada junte todas las imágenes procesadas en una unica carpeta
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/mazda_iconic_sp/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/dataset/test/mazda_iconic_sp/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    # TODO: Añadir función que al ser llamada junte todas las imágenes procesadas en una unica carpeta
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/mini_cooper_d/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/dataset/test/mini_cooper_d/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    # TODO: Añadir función que al ser llamada junte todas las imágenes procesadas en una unica carpeta
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/rover_p6_3500s/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/dataset/test/rover_p6_3500s/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    # TODO: Añadir función que al ser llamada junte todas las imágenes procesadas en una unica carpeta
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/volkswagen_up/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/dataset/test/volkswagen_up/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    
    
    

if __name__ == "__main__":
    main()

In [3]:
### VERSION 3

import os
from PIL import Image, ImageEnhance, ImageOps
import cv2
import numpy as np
from tqdm import tqdm

class ImageProcessor:
    def __init__(self, input_folder, output_folder):
        """
        Inicializa el procesador de imágenes.
        
        Args:
            input_folder (str): Ruta de la carpeta con imágenes originales
            output_folder (str): Ruta donde se guardarán las imágenes procesadas
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
        
    def create_output_dirs(self):
        """Crea los directorios necesarios para cada versión."""
        os.makedirs(self.output_folder, exist_ok=True)
        for i in range(1, 6):
            os.makedirs(os.path.join(self.output_folder, f"version_{i}"), exist_ok=True)
            os.makedirs(os.path.join(self.output_folder, f"version_{i}_mirror"), exist_ok=True)
    
    def apply_morning_filter(self, image):
        """
        Filtro de mañana soleada (9:00 AM).
        Partiendo de iluminación de estudio, ajusta para simular luz natural de mañana:
        - Aumenta temperatura de color ligeramente
        - Añade sombras suaves pero definidas
        - Mantiene buena exposición general
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (ligeramente cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajuste sutil de canales para luz de mañana
        b, g, r = cv2.split(img_array)
        # Reduce azules ligeramente
        b = cv2.convertScaleAbs(b, alpha=0.95, beta=0)
        # Aumenta rojos suavemente
        r = cv2.convertScaleAbs(r, alpha=1.05, beta=5)
        img_array = cv2.merge([b, g, r])
        
        # Ajuste general de contraste y brillo
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=5)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales de imagen
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.1)
        
        return image

    def apply_late_afternoon_filter(self, image):
        """
        Aplica filtro para simular luz de tarde (5:00 PM).
        Añade tonos cálidos y sombras más pronunciadas.
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (más cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Aumenta los tonos rojos y reduce los azules
        b, g, r = cv2.split(img_array)
        r = cv2.convertScaleAbs(r, alpha=1.15, beta=10)
        b = cv2.convertScaleAbs(b, alpha=0.9, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Ajusta el contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=0)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajusta saturación y contraste
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        
        return image

    def apply_cloudy_afternoon_filter(self, image):
        """
        Filtro de tarde nublada (5:00 PM).
        Simula luz difusa de día nublado:
        - Reduce el contraste general
        - Enfatiza los tonos medios
        - Añade una sutil dominante fría
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (ligeramente fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajustes de canales para luz difusa
        b, g, r = cv2.split(img_array)
        # Aumenta azules sutilmente
        b = cv2.convertScaleAbs(b, alpha=1.05, beta=5)
        # Reduce rojos ligeramente
        r = cv2.convertScaleAbs(r, alpha=0.95, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Reduce contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=0.9, beta=10)
        
        # Aplica desenfoque sutil para simular dispersión
        img_array = cv2.GaussianBlur(img_array, (3, 3), 0.5)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales para ambiente nublado
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(0.9)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.85)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.95)
        
        return image

    def apply_night_filter(self, image):
        """
        Filtro nocturno (10:00 PM).
        Simula iluminación artificial nocturna:
        - Reduce significativamente el brillo
        - Añade tono azulado de luz nocturna
        - Aumenta las sombras
        - Simula iluminación artificial
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajustes de canales para luz nocturna
        b, g, r = cv2.split(img_array)
        # Enfatiza azules
        b = cv2.convertScaleAbs(b, alpha=1.15, beta=10)
        # Reduce rojos
        r = cv2.convertScaleAbs(r, alpha=0.8, beta=-10)
        # Ajusta verdes
        g = cv2.convertScaleAbs(g, alpha=0.9, beta=-5)
        img_array = cv2.merge([b, g, r])
        
        # Reducción general de luminosidad
        img_array = cv2.convertScaleAbs(img_array, alpha=0.75, beta=-20)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales para ambiente nocturno
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.7)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.3)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.85)
        
        return image

    def create_mirror_version(self, image):
        """Crea versión espejo de la imagen."""
        return ImageOps.mirror(image)

    def save_image(self, image, version, filename, is_mirror=False):
        """Guarda la imagen procesada en el directorio correspondiente."""
        subfolder = f"version_{version}_mirror" if is_mirror else f"version_{version}"
        output_path = os.path.join(self.output_folder, subfolder, filename)
        image.save(output_path, quality=95, subsampling=0)

    def process_single_image(self, filename):
        """Procesa una única imagen y genera todas sus versiones."""
        input_path = os.path.join(self.input_folder, filename)
        
        try:
            # Abre la imagen original
            original = Image.open(input_path).convert('RGB')
            
            # Lista de funciones de procesamiento para cada versión
            processors = [
                lambda x: x,  # Versión 1: Original
                self.apply_morning_filter,  # Versión 2: Mañana soleada
                self.apply_late_afternoon_filter,  # Versión 3: Tarde
                self.apply_cloudy_afternoon_filter,  # Versión 4: Tarde nublada
                self.apply_night_filter,  # Versión 5: Noche
            ]
            
            # Procesa y guarda cada versión
            for version, processor in enumerate(processors, 1):
                # Procesa la imagen
                processed = processor(original.copy())
                
                # Guarda versión normal y espejo
                self.save_image(processed, version, (str(version) + "_" + filename))
                self.save_image(self.create_mirror_version(processed), version, ("m" + str(version) + "_" + filename), True)
                
        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")

    def process_all_images(self):
        """Procesa todas las imágenes en el directorio de entrada."""
        # Crea directorios de salida
        self.create_output_dirs()
        
        # Obtiene lista de imágenes
        images = [f for f in os.listdir(self.input_folder) 
                 if os.path.splitext(f)[1].lower() in self.valid_extensions]
        
        if not images:
            print("No se encontraron imágenes para procesar.")
            return
        
        print(f"Procesando {len(images)} imágenes...")
        
        # Procesa cada imagen con barra de progreso
        for filename in tqdm(images):
            self.process_single_image(filename)
            
        print("\n¡Procesamiento completado!")

    def consolidate_versions(self):
        """
        Consolida todas las versiones procesadas en una única carpeta dentro del directorio de salida.
        Las imágenes se mantienen separadas por cada conjunto procesado.
        """
        consolidated_folder = os.path.join(self.output_folder, "consolidated")
        print(f"\nConsolidando imágenes en: {consolidated_folder}")
        
        # Crea el directorio de consolidación
        os.makedirs(consolidated_folder, exist_ok=True)
        
        # Recorre todas las subcarpetas de versiones
        total_copied = 0
        for version in range(1, 6):
            for subfolder in [f"version_{version}", f"version_{version}_mirror"]:
                source_folder = os.path.join(self.output_folder, subfolder)
                
                # Verifica si existe la carpeta
                if os.path.exists(source_folder):
                    # Copia todas las imágenes de la subcarpeta
                    for filename in os.listdir(source_folder):
                        if any(filename.lower().endswith(ext) for ext in self.valid_extensions):
                            source_path = os.path.join(source_folder, filename)
                            dest_path = os.path.join(consolidated_folder, filename)
                            
                            try:
                                # Copia el archivo
                                with open(source_path, 'rb') as fsrc:
                                    with open(dest_path, 'wb') as fdst:
                                        fdst.write(fsrc.read())
                                total_copied += 1
                            except Exception as e:
                                print(f"Error copiando {filename}: {str(e)}")
        
        print(f"Se consolidaron {total_copied} imágenes exitosamente en {consolidated_folder}")

def main():
    # Procesa cada carpeta de auto individualmente
    # input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/ford_fiesta_zetec/"
    # output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/ford_fiesta_zetec/"
    # processor = ImageProcessor(input_folder, output_folder)
    # processor.process_all_images()
    # processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/mazda_iconic_sp/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mazda_iconic_sp/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/mini_cooper_d/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mini_cooper_d/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/rover_p6_3500s/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/rover_p6_3500s/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions()
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/volkswagen_up/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/volkswagen_up/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions()

if __name__ == "__main__":
    main()

Procesando 1794 imágenes...


100%|██████████| 1794/1794 [00:15<00:00, 115.51it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mazda_iconic_sp/consolidated
Se consolidaron 17940 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mazda_iconic_sp/consolidated
Procesando 1622 imágenes...


100%|██████████| 1622/1622 [00:13<00:00, 116.27it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mini_cooper_d/consolidated
Se consolidaron 16220 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mini_cooper_d/consolidated
Procesando 1579 imágenes...


100%|██████████| 1579/1579 [00:14<00:00, 110.13it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/rover_p6_3500s/consolidated
Se consolidaron 15790 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/rover_p6_3500s/consolidated
Procesando 1677 imágenes...


100%|██████████| 1677/1677 [00:18<00:00, 92.84it/s] 



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/volkswagen_up/consolidated
Se consolidaron 16770 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/volkswagen_up/consolidated
